In [1]:
%load_ext autoreload
%autoreload 1

In [2]:
from collections import defaultdict
import os, sys

mx_base = "/mnt/extproj/projekte/textmining/mxplore/"
model_base = mx_base + "spacy_models/"
mirexplore_base = mx_base + "/python/textmine/"
scai_base = mirexplore_base + "/scai_eval/"

if not mirexplore_base in sys.path:
    sys.path.insert(0, mirexplore_base)
    

import scispacy
import spacy
from spacy import displacy

#nlp = spacy.load('/mnt/d/spacy/models/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/')

from MirGeneRelCheck import SentenceRelationChecker, SentenceRelationClassifier
from mxutils.tmutils import normalize_gene_names
from collections import Counter
from lxml import etree
from sklearn.metrics import multilabel_confusion_matrix, classification_report
from itertools import chain, combinations


normGeneSymbols= normalize_gene_names(path=scai_base + "/hgnc_no_withdrawn.syn")
relationsFile = mirexplore_base + "allrels.csv"

In [3]:
nlp_weblg = spacy.load(model_base+"/en_core_web_lg-2.2.0/en_core_web_lg/en_core_web_lg-2.2.0/")
nlp = spacy.load(model_base+"/en_core_sci_lg-0.2.4/en_core_sci_lg/en_core_sci_lg-0.2.4/")
nlp_ent = spacy.load(model_base+"/en_ner_bionlp13cg_md-0.2.4/en_ner_bionlp13cg_md/en_ner_bionlp13cg_md-0.2.4")

In [4]:
import MirGeneRelCheck

In [5]:
doc = nlp_ent("Therefore, it seems that BANCR and miR-204-3p might act in a cumulative way on the development of the extrathyroidal invasion of the BRAFV600E-negative PTC.")

ents = doc.to_json()["ents"]

for ent in ents:
    print(doc.text[ent["start"]:ent["end"]], ent)

#doc.to_json()

BANCR {'start': 25, 'end': 30, 'label': 'GENE_OR_GENE_PRODUCT'}
miR-204-3p {'start': 35, 'end': 45, 'label': 'GENE_OR_GENE_PRODUCT'}
extrathyroidal {'start': 102, 'end': 116, 'label': 'MULTI-TISSUE_STRUCTURE'}
PTC {'start': 152, 'end': 155, 'label': 'CANCER'}


In [6]:

def analyse(testCase, dist=80):
    relCheck = MirGeneRelCheck.MirGeneRelCheck()

    doc = nlp(testCase[0])
    
    #print("conjuncts", doc.conjuncts)
    
    lWord = doc[testCase[1]]
    rWord = doc[testCase[2]]

    for t in doc:
        print(t.idx, t.i, t, t.pos_, t.dep_, t.head, t.head.idx, t.conjuncts)

    print(lWord, rWord)

    if not "mir" in str(lWord).lower() and not "let" in str(lWord).lower():
        tmp = lWord
        lWord = rWord
        rWord = tmp
        
    print(lWord, rWord)

    relClassifier = SentenceRelationClassifier(relationsFile)    
    checkResults = relCheck.checkRelation(doc, lWord, rWord, verbose=True, relClassifier=relClassifier.classify)
    print(checkResults)
    
    displacy.render(doc, style="dep", options={"compact": True, "distance": dist})

In [7]:
%aimport MirGeneRelCheck

testcase = (u"Among them, circ_002178 promoted PD-L1 expression via sponging miR-34a.", 5, 9)

analyse(testcase)

0 0 Among ADP prep promoted 24 ()
6 1 them PRON pobj Among 0 ()
10 2 , PUNCT punct promoted 24 ()
12 3 circ_002178 NOUN nsubj promoted 24 ()
24 4 promoted VERB ROOT promoted 24 ()
33 5 PD-L1 NOUN compound expression 39 ()
39 6 expression NOUN dobj promoted 24 ()
50 7 via ADP mark sponging 54 ()
54 8 sponging ADJ advcl promoted 24 ()
63 9 miR-34a NOUN dobj sponging 54 ()
70 10 . PUNCT punct promoted 24 ()
PD-L1 miR-34a
miR-34a PD-L1
Conjunctions ( conj )
Conjunctions ( nmod )
SDP sent: Among them, circ_002178 promoted PD-L1 expression via sponging miR-34a.
SDP     : [(PD-L1, 'NOUN', 'compound', False), (expression, 'NOUN', 'dobj', False), (promoted, 'VERB', 'ROOT', False), (sponging, 'ADJ', 'advcl', False), (miR-34a, 'NOUN', 'dobj', False)]
SDP DOBJ RULE
0 [Among, them, ,, circ_002178, promoted, PD-L1, expression, via, sponging, miR-34a] 10 0 Among them, circ_002178 promoted PD-L1 expression via sponging miR-34a.
0 [Among, them, ,, circ_002178, promoted, PD-L1, expression, via, sponging

Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


In [8]:
%aimport MirGeneRelCheck

testcase = (u"Functional association between miR 124a and CDK6 in medulloblastoma was established using luciferase assays.", 4, 6)

analyse(testcase)

0 0 Functional ADJ amod association 11 ()
11 1 association NOUN nsubjpass established 72 ()
23 2 between ADP case miR 31 ()
31 3 miR NOUN nmod association 11 ()
35 4 124a NOUN dep miR 31 (CDK6,)
40 5 and CCONJ cc 124a 35 ()
44 6 CDK6 NOUN conj 124a 35 (124a,)
49 7 in ADP case medulloblastoma 52 ()
52 8 medulloblastoma NOUN nmod association 11 ()
68 9 was AUX auxpass established 72 ()
72 10 established VERB ROOT established 72 ()
84 11 using VERB xcomp established 72 ()
90 12 luciferase NOUN compound assays 101 ()
101 13 assays NOUN dobj using 84 ()
107 14 . PUNCT punct established 72 ()
124a CDK6
CDK6 124a
Conjuncts [CDK6, 124a]
neighbour CDK6 in
neighbour CDK6 in
Conjunction before fc 124a
Conjuncts [124a, CDK6]
neighbour 124a and
neighbour 124a and
Conjunction before fc CDK6
Conjunctions ( conj )
124a [124a, CDK6]
achildren {CDK6}
achildren False
Conjuncts [association, miR]
neighbour association between
neighbour association between
Conjunction before fc miR
fc add children for miR


Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


In [9]:
%aimport MirGeneRelCheck

testcase = (u"Overexpression of mir-7 and mir-153 significantly reduces endogenous alpha-synuclein levels, whereas inhibition of mir-7 and mir-153 enhances translation of a luciferase construct bearing the alpha-s", 6, 10)

analyse(testcase)

0 0 Overexpression NOUN nsubj reduces 50 ()
15 1 of ADP case mir-7 18 ()
18 2 mir-7 NOUN nmod Overexpression 0 (mir-153,)
24 3 and CCONJ cc mir-7 18 ()
28 4 mir-153 NOUN conj mir-7 18 (mir-7,)
36 5 significantly ADV advmod reduces 50 ()
50 6 reduces VERB ROOT reduces 50 ()
58 7 endogenous ADJ amod levels 85 ()
69 8 alpha-synuclein ADJ amod levels 85 ()
85 9 levels NOUN dobj reduces 50 ()
91 10 , PUNCT punct reduces 50 ()
93 11 whereas SCONJ dep reduces 50 ()
101 12 inhibition NOUN nsubj enhances 133 ()
112 13 of ADP case mir-7 115 ()
115 14 mir-7 NOUN nmod inhibition 101 (mir-153,)
121 15 and CCONJ cc mir-7 115 ()
125 16 mir-153 NOUN conj mir-7 115 (mir-7,)
133 17 enhances VERB parataxis reduces 50 ()
142 18 translation NOUN dobj enhances 133 ()
154 19 of ADP case construct 170 ()
157 20 a DET det construct 170 ()
159 21 luciferase NOUN compound construct 170 ()


Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


170 22 construct NOUN nmod translation 142 ()
180 23 bearing VERB acl construct 170 ()
188 24 the DET det alpha-s 192 ()
192 25 alpha-s NOUN dobj bearing 180 ()
reduces ,
, reduces
Conjuncts [mir-153, mir-7]
neighbour mir-153 significantly
neighbour mir-153 significantly
Conjunction before fc mir-7
fc add children for mir-7
Conjuncts [mir-7, mir-153]
neighbour mir-7 and
neighbour mir-7 and
Conjunction before fc mir-153
Conjuncts [mir-153, mir-7]
neighbour mir-153 enhances
neighbour mir-153 enhances
Conjunction before fc mir-7
fc add children for mir-7
Conjuncts [mir-7, mir-153]
neighbour mir-7 and
neighbour mir-7 and
Conjunction before fc mir-153
Conunct merge check: mir-7 mir-7
mir-153 significantly reduces endogenous alpha-synuclein levels, whereas inhibition
Conjunctions ( conj )
mir-7 [of, mir-7, mir-153]
mir-7 [of, mir-7, mir-153]
Conjuncts [Overexpression, mir-7]
neighbour Overexpression of
neighbour Overexpression of
Conjunction before fc mir-7
fc add children for mir-7
Conjunct

In [10]:
analyse((u"The clinical relevance of our findings was evaluated in HIV-encephalitis (HIVE) brain samples in which decreased levels of MCP-2 and increased levels of mir-146a were observed, suggesting a role for mir-146a in the maintenance of HIV-mediated chronic inflammation of the brain.", 20, 25)
)

0 0 The DET det relevance 13 ()
4 1 clinical ADJ amod relevance 13 ()
13 2 relevance NOUN nsubjpass evaluated 43 ()
23 3 of ADP case findings 30 ()
26 4 our PRON nmod:poss findings 30 ()
30 5 findings NOUN nmod relevance 13 ()
39 6 was AUX auxpass evaluated 43 ()
43 7 evaluated VERB ROOT evaluated 43 ()
53 8 in ADP case samples 86 ()
56 9 HIV-encephalitis NOUN compound samples 86 ()
73 

Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


10 ( PUNCT punct HIVE 74 ()
74 11 HIVE NOUN appos HIV-encephalitis 56 ()
78 12 ) PUNCT punct HIVE 74 ()
80 13 brain NOUN compound samples 86 ()
86 14 samples NOUN nmod evaluated 43 ()
94 15 in ADP case which 97 ()
97 16 which PRON nmod observed 167 ()
103 17 decreased VERB amod levels 113 ()
113 18 levels NOUN nsubjpass observed 167 (levels,)
120 19 of ADP case MCP-2 123 ()
123 20 MCP-2 NOUN nmod levels 113 ()
129 21 and CCONJ cc levels 113 ()
133 22 increased VERB amod levels 143 ()
143 23 levels NOUN conj levels 113 (levels,)
150 24 of ADP case mir-146a 153 ()
153 25 mir-146a NOUN nmod levels 143 ()
162 26 were AUX auxpass observed 167 ()
167 27 observed VERB acl:relcl samples 86 ()
175 28 , PUNCT punct evaluated 43 ()
177 29 suggesting VERB xcomp evaluated 43 ()
188 30 a DET det role 190 ()
190 31 role NOUN dobj suggesting 177 ()
195 32 for ADP case mir-146a 199 ()
199 33 mir-146a NOUN nmod role 190 ()
208 34 in ADP case maintenance 215 ()
211 35 the DET det maintenance 215 ()
215 3

In [11]:
%aimport MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)

relCheck = MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

In [12]:
conjResult, conjs = relCheck.checkCommonConj(doc, mirword, geneword, verbose)
sdpPass, passive, negated = relCheck.checkSDP(doc, mirword, geneword, verbose)
compPass = relCheck.checkCompartments(doc, mirword, geneword, verbose)
sigPathway = relCheck.checkSurContext(doc, mirword, geneword, verbose)


In [13]:
sdpRes = relCheck._get_sdp_path(doc, mirword.i, geneword.i)
sdpRes

[(miR-106b, 'NOUN', 'nmod', False),
 (expression, 'NOUN', 'nsubjpass', False),
 (correlated, 'VERB', 'ccomp', True),
 (inhibit, 'VERB', 'conj', False),
 (translation, 'NOUN', 'dobj', False),
 (TβRII, 'NOUN', 'compound', False)]

In [14]:
%aimport MirGeneRelCheck

testCase = (u"We show here that miR-29a, -29b-1, and -9 can regulate BACE1 expression in vitro.", 4, 12)
testCase = (u"It has been predicted that miR-146a may target Tata Binding Protein (TBP).",5, 12)
testCase = (u"Our results showed that the expression of miR-106b was inversely correlated with TβRII protein levels and miR-106b can directly inhibit the TβRII translation in vitro.", 7, 21)
testCase = (u"The reduction of the affinity of miR-433 to the 3' UTR would result in increased FGF20 expression and upregulation of alpha-synuclein, which could in turn promote dopaminergic neurons degeneration.", 6, 16)
testCase = (u"These findings identify miR-29b as a novel posttranscriptional regulator of PGRN expression, raising the possibility that miR-29b or other miRNAs might be targeted therapeutically to increase hPGRN levels in some FTD patients.", 3,10)
relCheck = MirGeneRelCheck.MirGeneRelCheck()

doc = nlp(testCase[0])

mirword = doc[testCase[1]]
geneword = doc[testCase[2]]

verbose=False

from collections import Counter



In [15]:
def loadStems():
    
    print("Loading stems")
    allRels = []
    stem2class = {}

    renameClass = {
        'INT': 'NEU',
        "BOOST": "POS",
        "COMBINE": "NEU",
        "CHANGE": "NEU",
    }

    with open(mirexplore_base+"/allrels.csv") as fin:

        for line in fin:
            aline = line.strip().split(",", 1)
            aregSyns = aline[1].split(":")
            regSyns = []
            if len(aregSyns) > 1:
                regSyns = aregSyns[1].split("|")
            
            aline[1] = aline[1].split(":")[0]           

            regClass = aline[0]
            regStem = aline[1]

            if regClass in renameClass:
                regClass = renameClass[regClass]
                
            stem2class[regStem] = regClass
                
            for regSyn in regSyns:
                if not regSyn in stem2class:
                    stem2class[regSyn] = regClass


            allRels.append((regClass, regStem))

    allStems = [x for x in stem2class]
    
    return allStems, allRels, stem2class

allStems, allRels, stem2class = loadStems()


Loading stems


In [16]:
scaiBase = mirexplore_base + "/scai_corpus/"

base = "TRAIN"

if base == "TRAIN":
    scaiFile = "miRNA_train_fixed.xml"
    relsFile = "scai_train_rels.tsv"
elif base == "TEST":
    scaiFile = "miRNA_test_fixed.xml"
    relsFile = "scai_test_rels.tsv"
    
normGeneSymbols= normalize_gene_names(path=scai_base + "/hgnc_no_withdrawn.syn")

relChecker = SentenceRelationChecker(nlp)

correctIdentified = 0
incorrectIdentified = 0
totalChecks = 0
incorrectClass = Counter()


In [17]:
%aimport MirGeneRelCheck

def loadRelations(relsFile):
    reldir2new = {}
    reldir2new["TARGET"] = "NEU"
    reldir2new["INVCORR"] = "NEU"
    reldir2new["REGULATE"] = "NEU"
    relClasses = {}
    with open(os.path.join(scai_base, relsFile)) as fin:
        
        print("Loading relations", relsFile)
        
        for lidx, line in enumerate(fin):

            if lidx == 0:
                continue

            line = line.strip().split("\t")

            interactionID = line[0].strip()
            relType = line[1].strip()
            relRDir = line[2].strip()
            relIndirect = line[3].strip()
            relPassive = line[4].strip()

            if relRDir in reldir2new:
                relRDir = reldir2new[relRDir]

            relClasses[interactionID] = (relType, relRDir, relIndirect, relPassive)
            
        print("Interactions", len(relClasses))
        print("Non NA Interactions", len([x for x in relClasses if not relClasses[x][0] == "NA"]))
            
    return relClasses

def all_subsets(ss):
    
    return chain(*map(lambda x: combinations(ss, x), range(0, len(ss)+1)))

def runCheck(numelems, base, relation_ids=None, original=False):
    relChecker = SentenceRelationChecker(nlp, nlp_ent=nlp_ent)
    relClassifier = SentenceRelationClassifier(relationsFile)
    
    if base == "TRAIN":
        scaiFile = "miRNA_train_fixed.xml"
        relsFile = "scai_train_rels.tsv"
    elif base == "TEST":
        scaiFile = "miRNA_test_fixed.xml"
        relsFile = "scai_test_rels.tsv"
        
    if original:
        if base == "TRAIN":
            scaiFile = "miRNA-Train-Corpus.xml"
        elif base == "TEST":
            scaiFile = "miRNA-Test-Corpus.xml"
    
    relClasses = loadRelations(relsFile)
    
    check2results = {}
    checkSubsets = [x for x in all_subsets(relClassifier.major_checks)]
    checkSubsets = [checkSubsets[-1]]
    
    print("Will test {} subsets.".format(len(checkSubsets)))
    
    for subset_checks in checkSubsets:
    
        print("Testing checks", subset_checks)
        
        #relClassifier.active_checks = subset_checks

        #allStems, allRels, stem2class = loadStems()

        with open(os.path.join(scaiBase, scaiFile), 'r') as fin:
            tree = etree.parse(fin)
            root = tree.getroot()
            scaiPairs = []
            totalChecks = 0
            correctIdentified = 0
            incorrectIdentified = 0

            incorrectClassified = 0
            correctClassified = 0
            totalClassified = 0
            correctInteractClassified = 0
            totalValidClassified = 0
            totalClassifiable = 0
            totalMirGeneDownClassified = 0

            errorByDetect = Counter()
            classifiedByDetect = Counter()
            elemCaseCounter = Counter()
            incorrectClass = Counter()
            relationFalseCounter = Counter()

            classifyTrue = []
            classifyPred = []

            for elem in root.findall(".//document"):

                pmid = elem.attrib['origId']

                for sentElem in elem:

                    allEntities = sentElem.findall(".//entity")
                    allPairs = sentElem.findall(".//pair")

                    sentText = sentElem.attrib["text"]

                    entId2elem = {}

                    for entity in allEntities:
                        entId = entity.attrib['id']
                        entText = entity.attrib['text']
                        entType = entity.attrib['type']
                        entOffset = tuple([int(x) for x in entity.attrib['charOffset'].split("-")])

                        if entType in ["Specific_miRNAs", "Genes/Proteins"]:

                            if "Genes" in entType:
                                if entText in normGeneSymbols:
                                    entText = normGeneSymbols[entText]
                                elif entText.upper() in normGeneSymbols:
                                    gene = normGeneSymbols[entText.upper()]
                            else:
                                try:
                                    entText = miRNA(entText).getStringFromParts([miRNAPART.MATURE, miRNAPART.ID, miRNAPART.PRECURSOR])
                                except:
                                    pass

                            entTuple = (entText, entType, (entOffset[0], entOffset[1]+1))
                            entId2elem[entId] = entTuple


                            sentEntText = sentText[entTuple[2][0]:entTuple[2][1]]

                    for pair in allPairs:

                        validInteraction = pair.attrib['interaction'].lower() == "true"
                        pairE1 = pair.attrib['e1']
                        pairE2 = pair.attrib['e2']
                        relationID = pair.attrib["id"]
                        if not relation_ids is None:
                            if not relationID in relation_ids:
                                continue

                        #if pairInt == 'true':
                        if pairE1 in entId2elem and pairE2 in entId2elem:

                            totalChecks += 1

                            e1 = entId2elem[pairE1]
                            e2 = entId2elem[pairE2]

                            if not e1[1] in ["Specific_miRNAs"]:

                                tmp=e1
                                e1=e2
                                e2=tmp


                            relRes = relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=False
                                                                )


                            fullsentence = relRes['full_sentence']
                            acceptInteraction = relRes['accept_relation']
                            
                            

                            if not acceptInteraction == validInteraction:

                                incorrectIdentified += 1
                                
                                print()
                                print()
                                print(relationID)
                                print("IS:", acceptInteraction)
                                print("SB:", validInteraction)
                                
                                print("Repeat Analysis")
                                relResV=relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=True
                                                                )
                                
                                print(relResV)
                                
                                for check in ['conj', 'conj_nmod', 'sdp', 'compartment', 'compartment_nmod', 'context', 'entity']:
                                    if relResV["check_results"][check] == False:
                                        relationFalseCounter[check] += 1

                                
                            else:
                                correctIdentified += 1

                            elemCaseCounter[(validInteraction, acceptInteraction)]+=1

                            totalClassified += 1

                            if validInteraction:# and acceptInteraction:
                                #print(relRes["check_results"])   
                                totalValidClassified += 1

                                foundClasses = relRes["check_results"]["classification"] #{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'regulation_keys': set()}
                                foundTuple = (foundClasses["interaction_dir"], foundClasses["regulation_dir"])                                
                                
                                testTuple = relClasses[relationID]

                                classifyTrue.append((testTuple[0], testTuple[1]))
                                classifyPred.append((foundTuple[0], foundTuple[1]))

                                if testTuple[0] == "MIR_GENE" and testTuple[1] == "DOWN":
                                    totalMirGeneDownClassified += 1

                                classifiedByDetect[foundClasses["reg_detect"]] += 1

                                if testTuple[0] != foundTuple[0] or testTuple[1] != foundTuple[1]:


                                    #print(sentText)
                                    #print(e1, e2)
                                    #print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                    #print(testTuple)

                                    print()
                                    print()
                                    print()
                                    
                                    print(relationID)
                                    print("IS: ", foundTuple)
                                    print("SB: ", testTuple)
                                    print(relRes)
                                    
                                    relResV=relChecker.check_sentence(sentText
                                                                , {"entity_type": "mirna", "entity_type_token": "e1", "entity_location": e1[2]}
                                                                , {"entity_type": "gene", "entity_type_token": "e2", "entity_location": e2[2]}
                                                                , fix_special_chars=False
                                                                , relClassifier=relClassifier.classify, verbose=True
                                                                )
                                
                                    print(relResV)
                                    
                                    print()
                                    print()
                                    print()

                                    errorByDetect[foundClasses["reg_detect"]] += 1
                                    
                                    

                                    incorrectClassified += 1



                                    #numelems -= 1
                                    #if numelems == 0:
                                    #    return
                                else:
                                    
                                    if foundClasses["reg_detect_major"] == "return":
                                        print(foundClasses)
                                        print(sentText)
                                        print(e1, e2)
                                        print(foundTuple, foundClasses["reg_detect"], "passive?", foundClasses["passive"])
                                        print(testTuple)

                                        print()
                                        print()
                                        print()
                                    correctClassified += 1

                                    if acceptInteraction:
                                        correctInteractClassified += 1



            def printStats(outfile):
                print("Total:     ", totalChecks, file=outfile)
                print("Correct:   ", correctIdentified, correctIdentified/totalChecks, file=outfile)
                print("Incorrect: ", incorrectIdentified, incorrectIdentified/totalChecks, file=outfile)
                print("classes", incorrectClass, file=outfile)
                
                precision = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(True, False)])
                recall = elemCaseCounter[(True, True)] / (elemCaseCounter[(True, True)]+elemCaseCounter[(False, True)])

                f1 = 2* precision * recall / (precision+recall)

                specificity = elemCaseCounter[(False, False)] / (elemCaseCounter[(True, False)] + elemCaseCounter[(False, False)])

                print("precision", precision, file=outfile)
                print("recall", recall, file=outfile)
                print("specificity", specificity, file=outfile)
                print("f1", f1, file=outfile)
                
                print("relationFalseCounter", relationFalseCounter, file=outfile)

                print("Correct classified:     ", correctClassified, correctClassified/totalClassified, correctClassified/totalValidClassified,file=outfile)
                print("Incorrect classified:   ", incorrectClassified, incorrectClassified/totalClassified, incorrectClassified/totalValidClassified,file=outfile)
                print("Random classified: ", totalMirGeneDownClassified, totalMirGeneDownClassified / totalClassified, totalMirGeneDownClassified/totalValidClassified, file=outfile)
                print(errorByDetect)

                print("Correct interaction&classified:     ", correctInteractClassified, correctInteractClassified/totalClassified, correctInteractClassified/totalValidClassified,file=outfile)

                print("Classified by: ", classifiedByDetect)

                lClassifyTrue = ["_".join(x) for x in classifyTrue]
                lClassifyPred = ["_".join(x) for x in classifyPred]

                allLabels = sorted(set(lClassifyTrue+lClassifyPred))
                matrix = multilabel_confusion_matrix(lClassifyTrue, lClassifyPred, labels=allLabels)
                print(matrix)
                print(classification_report(lClassifyTrue,lClassifyPred))
                
                return classification_report(lClassifyTrue, lClassifyPred, output_dict=True)

            rep = printStats(sys.stdout)
            check2results[subset_checks] = rep
        #printStats(sys.stderr)
    return check2results
        


In [18]:
%aimport MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

train_results = runCheck(-1, "TRAIN")

Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


Loading relations scai_train_rels.tsv
Interactions 397
Non NA Interactions 194
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')


miRNA-corp.d1.s8.p0
IS: False
SB: True
Repeat Analysis
add special case microRNA-200a
add special case E-cadherin
microRNA-200a 1 E-cadherin 9
(u"Downregulated microRNA-200a in meningiomas promotes tumor growth by reducing E-cadherin and activating the Wnt/beta-catenin signaling pathway.", 1, 9)
geneword E-cadherin mirword microRNA-200a
Conjuncts [activating, reducing]
neighbour activating the
neighbour activating the
Conjunction before fc reducing
fc add children for reducing
Conjuncts [reducing, activating]
neighbour reducing E-cadherin
neighbour reducing E-cadherin
Conjunction before fc activating
fc add children for activating
Conjunctions ( conj )
reducing [reducing, E-cadherin, activating, Wnt/beta-catenin, signaling, pathway]
Pathway False
Conjuncts [microRNA-200a, meningiomas]
neighbour microRNA-200a in
neighbour mic

In [19]:
%aimport MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

test_results = runCheck(-1, "TEST")

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv


{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines. 
('miRNA-128', 'Specific_miRNAs', (22, 31)) ('APOBEC3C', 'Genes/Proteins', (99, 103))
('MIR_GENE', 'NEU') return passive? False
('MIR_GENE', 'NEU', 'FALSE', 'FALSE')



{'regulation_dir': 'NEU', 'interaction_dir': 'MIR_GENE', 'passive': False, 'reg_detect': 'return', 'reg_detect_major': 'return'}
Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines. 
('miRNA-128', 'Specific_miRNAs', (22, 31)) ('ANGPTL6', 'Genes/Proteins', (105, 112))
('MIR_GENE', 'NEU') retur

In [20]:
import pickle

outDirectory = mx_base + "/scai_eval/"

with open(os.path.join(outDirectory, "scai_test_f1.pickle"), "wb") as fout:
    pickle.dump(test_results, fout)

In [21]:
%aimport MirGeneRelCheck

relClassifier = SentenceRelationClassifier(relationsFile)

test_results_original = runCheck(-1, "TEST", original=True)

Loading relations scai_test_rels.tsv
Interactions 232
Non NA Interactions 141
Will test 1 subsets.
Testing checks ('compartment', 'between', 'counts', 'return')


Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv
Setting relfile /mnt/extproj/projekte/textmining/miRExplore/python/allrels.csv




miRNA-corp.d1.s1.p0
IS: True
SB: False
Repeat Analysis
add special case miRNA-128
add special case ARP5
miRNA-128 3 ARP5 18
(u"Addition of exogenous miRNA-128 to CRL-1690 and CRL-2610 GBM cell lines (a) restored 'homeostatic' ARP5 (ANGPTL6), Bmi-1 and E2F-3a expression, and (b) significantly decreased the proliferation of CRL-1690 and CRL-2610 cell lines.", 3, 18)
geneword ARP5 mirword miRNA-128
Conjuncts [lines, CRL-1690]
neighbour lines (
neighbour lines a
Conjunction before fc CRL-1690
fc add children for CRL-1690
Conjuncts [CRL-1690, lines]
neighbour CRL-1690 and
neighbour CRL-1690 and
Conjunction before fc lines
fc add children for lines
fc add children for lines
fc add children for lines
Conjuncts [decreased, restored]
neighbour decreased the
neighbour decreased the
Conjunction before fc restored
fc add children for restored
fc add children for restored
Conjuncts [Bmi-1, E2F-3a, ARP5]
neighbour Bmi-1 and
neighbour Bmi-1 and
neighbour E2F-3a expression
neighbour E2F-3a expressio

/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/j/joppich/.local/lib/python3.6/si